IMPORT LIBRARY

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

IMPORT DATASET

In [2]:
#  path ke direktori data train, validation, dan test
train_dir = "dataset\\train"
val_dir = "dataset\\val"
test_dir = "dataset\\test"

MODELLING CNN

In [3]:
from keras.optimizers import Adam
from keras.layers import Dense, Flatten

# Adjust the learning rate
optimizer = Adam(learning_rate=0.0001)

# Create a more complex model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


PREPROCESSING CNN

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Inisialisasi ImageDataGenerator dengan augmentasi
train_datagen = ImageDataGenerator(
    rotation_range=20,      # Rotasi 20 derajat
    zoom_range=0.2,         # Zoom 20%
    horizontal_flip=True,    # Flip horizontal
    rescale=1./255  # Normalization: mengubah skala menjadi [0, 1]
)


TRAINING DAN VALIDASI CNN

In [4]:

# Set generator untuk data train
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Ukuran gambar 
    batch_size=32,
    class_mode='binary',  # Set class_mode menjadi 'binary'
    classes=['normal', 'pneumonia']  # Specify the class labels
)

# Set generator untuk data validation
val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Set class_mode to 'binary'
    classes=['normal', 'pneumonia']  # Specify the class labels
)

# Latih model menggunakan generator
history = model.fit(
    train_generator,
    steps_per_epoch= len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps= len(val_generator)
)

model.save('pneumonia_cnn_epoch10.h5')

Found 5000 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10


157/157 [==============================] - 295s 2s/step - loss: 0.3788 - accuracy: 0.8308 - val_loss: 41.4004 - val_accuracy: 0.8622
Epoch 2/10
157/157 [==============================] - 354s 2s/step - loss: 0.2401 - accuracy: 0.9010 - val_loss: 54.7310 - val_accuracy: 0.8686
Epoch 3/10
157/157 [==============================] - 355s 2s/step - loss: 0.2011 - accuracy: 0.9210 - val_loss: 75.6835 - val_accuracy: 0.8429
Epoch 4/10
157/157 [==============================] - 301s 2s/step - loss: 0.2021 - accuracy: 0.9180 - val_loss: 54.2976 - val_accuracy: 0.8734
Epoch 5/10
157/157 [==============================] - 173s 1s/step - loss: 0.1961 - accuracy: 0.9198 - val_loss: 65.1363 - val_accuracy: 0.8590
Epoch 6/10
157/157 [==============================] - 182s 1s/step - loss: 0.1765 - accuracy: 0.9322 - val_loss: 47.8559 - val_accuracy: 0.9071
Epoch 7/10
157/157 [==============================]

c:\Users\jasonl\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


TESTING MODEL CNN

In [6]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import csv

# Load model yang sudah disimpan
model = load_model('pneumonia_cnn_epoch10.h5')

# Direktori tempat menyimpan file gambar untuk pengujian
test_dir = 'dataset\\test'

# List nama kelas (subfolder)
class_names = os.listdir(test_dir)

# Buat file CSV
csv_file_path = 'result_cnn_v3.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Tulis header ke file CSV
    csv_writer.writerow(['File', 'Predicted Class', 'Probabilities'])

    # Loop untuk setiap kelas dalam direktori pengujian
    for class_name in class_names:
        class_dir = os.path.join(test_dir, class_name)
        
        # List file dalam subfolder
        test_files = os.listdir(class_dir)

        # Loop untuk setiap file dalam subfolder
        for test_file in test_files:
            # Buat path lengkap untuk file gambar
            img_path = os.path.join(class_dir, test_file)

            # Muat gambar dan sesuaikan ukurannya dengan format yang diharapkan oleh model
            img = image.load_img(img_path, target_size=(224, 224))

            # Konversi gambar menjadi array numpy
            img_array = image.img_to_array(img)

            # Tambahkan dimensi batch karena model membutuhkan batch input
            img_array = np.expand_dims(img_array, axis=0)

            # Lakukan prediksi menggunakan model
            predictions = model.predict(img_array)

            # Ambil label kelas berdasarkan nilai probabilitas tertinggi
            predicted_class = 'normal' if predictions[0][0] < 0.5 else 'pneumonia'

            # Tampilkan hasil prediksi
            print(f'File: {test_file}, Predicted Class: {predicted_class}, Probabilities: {predictions}')

            # Tulis hasil prediksi ke dalam file CSV
            csv_writer.writerow([test_file, predicted_class, predictions[0][0]])

print(f'Hasil prediksi disimpan dalam file CSV: {csv_file_path}')


1/1 [==============================] - 0s 352ms/step
File: normal-IM-0115-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 44ms/step
File: normal-IM-0117-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 34ms/step
File: NORMAL-IM-0119-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 51ms/step
File: NORMAL-IM-0122-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 51ms/step
File: NORMAL-IM-0125-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 35ms/step
File: NORMAL-IM-0127-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 44ms/step
File: NORMAL-IM-0180-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 35ms/step
File: NORMAL-IM-0182-0001.jpeg

MODELLING FNN

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

# Inisialisasi model FNN
model_fnn = Sequential()

# Menambahkan layer input
model_fnn.add(Flatten(input_shape=(224, 224, 3)))

# Menambahkan layer tersembunyi
model_fnn.add(Dense(128, activation='relu'))

# Menambahkan layer output
model_fnn.add(Dense(1, activation='sigmoid'))

# Inisialisasi optimizer adam
optimizer = Adam(learning_rate=0.0001)

# Kompilasi model
model_fnn.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Tampilkan ringkasan model
model_fnn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 150528)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               19267712  
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 19267841 (73.50 MB)
Trainable params: 19267841 (73.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


PREPROCESSING FNN

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Inisialisasi ImageDataGenerator dengan augmentasi
train_datagen = ImageDataGenerator(
    rotation_range=20,      # Rotasi 20 derajat
    zoom_range=0.2,         # Zoom 20%
    horizontal_flip=True,    # Flip horizontal
    rescale=1./255  # Normalization: mengubah skala menjadi [0, 1]
)

TRAINING DAN VALIDASI MODEL FNN

In [8]:
# Set generator untuk data train
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Ukuran gambar 
    batch_size=32,
    class_mode='binary',  # Set class_mode menjadi 'binary'
    classes=['normal', 'pneumonia']  # Specify the class labels
)

# Set generator untuk data validation
val_generator = ImageDataGenerator().flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',  # Set class_mode to 'binary'
    classes=['normal', 'pneumonia']  # Specify the class labels
)

# Latih model menggunakan generator
history = model_fnn.fit(
    train_generator,
    steps_per_epoch= len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps= len(val_generator)
)

model_fnn.save('pneumonia_fnn_epoch10.h5')

Found 5000 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10
157/157 [==============================] - 232s 1s/step - loss: 0.5705 - accuracy: 0.8132 - val_loss: 46.5853 - val_accuracy: 0.8205
Epoch 2/10
157/157 [==============================] - 304s 2s/step - loss: 0.4097 - accuracy: 0.8294 - val_loss: 107.1392 - val_accuracy: 0.7500
Epoch 3/10
157/157 [==============================] - 295s 2s/step - loss: 0.3305 - accuracy: 0.8650 - val_loss: 48.6343 - val_accuracy: 0.8301
Epoch 4/10
157/157 [==============================] - 236s 2s/step - loss: 0.3385 - accuracy: 0.8616 - val_loss: 48.5302 - val_accuracy: 0.8365
Epoch 5/10
157/157 [==============================] - 138s 882ms/step - loss: 0.3267 - accuracy: 0.8594 - val_loss: 133.1373 - val_accuracy: 0.7372
Epoch 6/10
157/157 [==============================] - 197s 1s/step - loss: 0.3048 - accuracy: 0.8666 - val_loss: 48.8496 - val_accuracy: 0.8317
Epoch 7/10
157/157 [============================

TESTING MODEL FNN

In [10]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import csv

# Load model yang sudah disimpan
model = load_model('pneumonia_fnn_epoch10.h5')

# Direktori tempat menyimpan file gambar untuk pengujian
test_dir = 'dataset\\test'

# List nama kelas (subfolder)
class_names = os.listdir(test_dir)

# Buat file CSV
csv_file_path = 'result_fnn_v3.csv'
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Tulis header ke file CSV
    csv_writer.writerow(['File', 'Predicted Class', 'Probabilities'])

    # Loop untuk setiap kelas dalam direktori pengujian
    for class_name in class_names:
        class_dir = os.path.join(test_dir, class_name)
        
        # List file dalam subfolder
        test_files = os.listdir(class_dir)

        # Loop untuk setiap file dalam subfolder
        for test_file in test_files:
            # Buat path lengkap untuk file gambar
            img_path = os.path.join(class_dir, test_file)

            # Muat gambar dan sesuaikan ukurannya dengan format yang diharapkan oleh model
            img = image.load_img(img_path, target_size=(224, 224))

            # Konversi gambar menjadi array numpy
            img_array = image.img_to_array(img)

            # Tambahkan dimensi batch karena model membutuhkan batch input
            img_array = np.expand_dims(img_array, axis=0)

            # Lakukan prediksi menggunakan model
            predictions = model.predict(img_array)

            # Ambil label kelas berdasarkan nilai probabilitas tertinggi
            predicted_class = 'normal' if predictions[0][0] < 0.5 else 'pneumonia'

            # Tampilkan hasil prediksi
            print(f'File: {test_file}, Predicted Class: {predicted_class}, Probabilities: {predictions}')

            # Tulis hasil prediksi ke dalam file CSV
            csv_writer.writerow([test_file, predicted_class, predictions[0][0]])

print(f'Hasil prediksi disimpan dalam file CSV: {csv_file_path}')


1/1 [==============================] - 0s 76ms/step
File: normal-IM-0115-0001.jpeg, Predicted Class: pneumonia, Probabilities: [[1.]]
1/1 [==============================] - 0s 47ms/step
File: normal-IM-0117-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 45ms/step
File: NORMAL-IM-0119-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 47ms/step
File: NORMAL-IM-0122-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 43ms/step
File: NORMAL-IM-0125-0001.jpeg, Predicted Class: pneumonia, Probabilities: [[1.]]
1/1 [==============================] - 0s 48ms/step
File: NORMAL-IM-0127-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 51ms/step
File: NORMAL-IM-0180-0001.jpeg, Predicted Class: normal, Probabilities: [[0.]]
1/1 [==============================] - 0s 49ms/step
File: NORMAL-IM-0182-0001